# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [1]:
import requests as re
import os
import json
from IPython.display import JSON

##### Useful JSON functions:
print(json.dumps(res.json(), indent=4, sort_keys=True))

JSON(res.json())

## Demo

In [2]:
# don't forget package os
app_id = os.environ["TransportForLondonPrimaryKey"]
app_key = os.environ["TransportForLondonSecondaryKey"]
url_append = f'?app_id={app_id}&app_key={app_key}' 

In [3]:
#print(url_append)

In [3]:
# URL
root_endpoint = 'https://api.tfl.gov.uk/' 
api_method = "AirQuality"
endpoint = root_endpoint + api_method

In [14]:
# We send the request to the API
# NOTE: if you don't have your APP_KEY, run this without the url_append

res = re.get(endpoint)

In [15]:
# We can check if the request was successful
res.status_code

200

In [128]:
# We can export the information that was returned using method .json()

#res.json()

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [17]:
# Structure of res.json():    dictionary{ list [ dictionary {} ] }

res.json()['currentForecast'][1]['forecastSummary']

'Low air pollution forecast valid from Friday 23 July to end of Friday 23 July GMT'

## Task
What are the different modes of transport which are operated by Transfer for London? How many of modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [75]:
def JourneyModes():
    api_method = 'Line/Meta/Modes'
    endpoint = root_endpoint + api_method
    
    res = re.get(endpoint)
    
    TransportModes = []
    
    for dct in res.json():
        TransportModes.append(dct['modeName'])
    
    return TransportModes

#res.json()
print(JourneyModes())
print("Number of different tranport modes:", len(JourneyModes()))

['bus', 'cable-car', 'coach', 'cycle', 'cycle-hire', 'dlr', 'interchange-keep-sitting', 'interchange-secure', 'national-rail', 'overground', 'replacement-bus', 'river-bus', 'river-tour', 'taxi', 'tflrail', 'tram', 'tube', 'walking']
Number of different tranport modes: 18


## Task
How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [24]:
def BikePointsFunc():
    api_method = 'BikePoint'
    endpoint = root_endpoint + api_method
    
    res = re.get(endpoint)
    
    BikePointsNum = []
    EmptyDocks = 0
    FullDocks = 0
    NumBikes = 0
    
    for dct in res.json():
        BikePointsNum.append(dct['id']) # Determine number of BikePoints.
        
        for addPropListElement in dct['additionalProperties']: # Determine number of empty and full BikePoint docks at each BikePoint.
            if addPropListElement['key'] == 'NbEmptyDocks':
                EmptyDocks += int(addPropListElement['value'])
            if addPropListElement['key'] == 'NbDocks':
                FullDocks += int(addPropListElement['value'])
            if addPropListElement['key'] == 'NbBikes':
                NumBikes += int(addPropListElement['value'])
    
    # nbDocks - (nbBikes + nbSpaces) != 0
    BrokenDocks = FullDocks - (NumBikes + EmptyDocks)
    
    print('Number of BikePoint Stations:', len(BikePointsNum))
    print(f'Number of docks in all BikePoints: {EmptyDocks + FullDocks} ({EmptyDocks} empty + {FullDocks} full)')    
    #print(f'Number of bikes: {NumBikes}')
    print(f'Number of broken docks: {BrokenDocks} (~{round(927/32245*100)}%)')
    print(f'Average number of docks at each BikePoint: {round((EmptyDocks + FullDocks)/len(BikePointsNum))}')
    #print(res.json()[0]['additionalProperties'][0]['key'])
    #JSON(res.json())
    
    return #JSON(res.json())

BikePointsFunc()

Number of BikePoint Stations: 782
Number of docks in all BikePoints: 32241 (11457 empty + 20784 full)
Number of broken docks: 927 (~3%)
Average number of docks at each BikePoint: 41


## Task
How many tube and bus lines are in London? Print names of all tube lines.

In [91]:
def LineInfo(modeName):
    api_method = 'Line/Mode/'
    endpoint = root_endpoint + api_method + modeName
    
    res = re.get(endpoint)    
    
    itemNum = 0
    
    if modeName == 'tube':
        tubeNames = []
        for dct in res.json():
            itemNum += 1
            tubeNames.append(dct['name'])
    else:
        for dct in res.json():
            itemNum += 1
    
    print(f'Number of {modeName} lines: {itemNum}')
    if modeName == 'tube':
        print(f'Tube line name list: {tubeNames}')
    
    #print(json.dumps(res.json(), indent=4, sort_keys=True))
        
    return

LineInfo('bus')
LineInfo('tube')

Number of bus lines: 676
Number of tube lines: 11
Tube line name list: ['Bakerloo', 'Central', 'Circle', 'District', 'Hammersmith & City', 'Jubilee', 'Metropolitan', 'Northern', 'Piccadilly', 'Victoria', 'Waterloo & City']


## Task
How many station has `victoria` line?

In [104]:
# Get a list of the stations that serve the given line id

def StationServingGivenLine(Line):
    api_method = 'Line/'
    endpoint = root_endpoint + api_method + Line + '/StopPoints'
    
    res = re.get(endpoint)    
    
    lineQuantity = 0
    
    for dct in res.json():
        lineQuantity += 1
        
    print(f'Number of stations serving the {Line} line: {lineQuantity}')
    #print(json.dumps(res.json(), indent=4, sort_keys=True))
        
    return

StationServingGivenLine('victoria')

Number of stations serving the victoria line: 16


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

In [127]:
def JourneyPlanner(Start,Dest,Mode):
    api_method = '/Journey/JourneyResults/'
    endpoint = root_endpoint + api_method + Start + '/to/' + Dest
    
    res = re.get(endpoint,  #use endpoint and all the parameters we need in a dictionary
              params={'mode': Mode})
    
    journeyTimeList = []
    
    for journey in res.json()['journeys']:
        journeyTimeList.append(journey['duration'])
    
    print(f'Fastest {Mode} journey: {min(journeyTimeList)} minutes')
    
    return 

JourneyPlanner('51.46962511264,-0.44968473098', '51.50599630145,-0.07502752221','bus')
JourneyPlanner('51.46962511264,-0.44968473098', '51.50599630145,-0.07502752221','tube')

Fastest bus journey: 104 minutes
Fastest tube journey: 81 minutes
